In [3]:
# Import statements
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

#visualizations import statements
import panel as pn
pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
import matplotlib.pyplot as plt
import holoviews as hv
import os
from dotenv import load_dotenv

In [9]:
# read the house_prices_msa.csv. this file will be used for sale price data
file_name='msa_pop_by_year.csv'
csv_file=Path(f"../Data/{file_name}")
census_df= pd.read_csv(csv_file)

# I drop rows that i do not need
census_df=census_df.loc[2:430]

# I rename field that will be used as column
census_df.iloc[0,0]='Identifier'

#grab the first row for the header
new_header = census_df.iloc[0] 

#take the data less the header row
census_df = census_df[1:] 

#set the header row as the df header and reset index
census_df.columns = new_header 
census_df.reset_index(inplace=True)

# Delete name of existing index
census_df.columns.name = None

# Drop unnecessary columns and rows
census_df.drop(columns=['index','Census','Estimates Base'],inplace=True) 

census_df.drop([1],inplace=True) 
# census_df.drop(['.In Metropolitan Statistical Area'],inplace=True) 

# Delete Rows containing Metro Divisions
census_df['Type']= census_df['Identifier'].str[-14:]
census_df.drop(census_df[census_df['Type']=='Metro Division'].index, inplace = True) 
census_df.drop(census_df[census_df['Identifier']=='.In Metropolitan Statistical Area'].index, inplace = True) 
census_df.drop(census_df[census_df['Identifier']=='.'].index, inplace = True) 

# Split MSA from text string - Used to Create City, State and 
census_df[['City','extra']] = census_df['Identifier'].str.split(", ", n = 1, expand=True) 
census_df['State'] = census_df['extra'].str[0:2]
census_df[['City1','extra1']] = census_df['City'].str.split(".", n = 1, expand=True) 
census_df[['City2','extra2']] = census_df['extra1'].str.split(".", n = 1, expand=True) 

# Rename columns and delete the ones we do not use
# Clean and rename
census_df.rename(columns={'City':'drop', 'City2':'City'}, inplace= True)
census_df.drop(columns=['drop','City1','extra','extra1','extra2'],inplace=True) 

# Final Split MSA from text string
census_df[['City','extra']] = census_df['City'].str.split("-", n = 1, expand=True) 
census_df.drop(columns=['extra'],inplace=True) 

# census_df['MSA'] = census_df['MSA'].str[1:]
census_df['MSA'] = census_df['City']+', '+census_df['State']

# census_df=census_df.set_index('')
census_df.sort_values(by='MSA')

# Fill NANs for Puerto Rico and United States
census_df.iloc[0,-1]='United States'
census_df.MSA[census_df.City=='United States'] = 'United States'
census_df.MSA[census_df.City=='Puerto Rico'] = 'Puerto Rico'

# Create data farme for census statistics (CAGR, Last, others by MSA)
census_stats_df=pd.DataFrame(census_df[['MSA','City','State']])
census_stats_df=census_stats_df.set_index('MSA')


# Final clean up of data frame census_df - Drop unused columns and re Arrange Columns
census_df.drop(columns=['Identifier','Type','City','State'], inplace=True)
census_df=census_df.set_index('MSA')
census_df=census_df.transpose()

# Delete name of existing index
census_df.columns.name = None
census_df.head(5)

,United States,"Abilene, TX","Akron, OH","Albany, GA","Albany, OR","Albany, NY","Albuquerque, NM","Alexandria, LA","Allentown, PA","Altoona, PA",...,"Yuma, AZ",Puerto Rico,"Aguadilla, PR","Arecibo, PR","Guayama, PR","Mayagüez, PR","Ponce, PR","San Germán, PR","San Juan, PR","Yauco, PR"
2010,"309,321,666","165,585","703,031","154,145","116,891","871,082","889,558","154,096","821,923","127,045",...,"197,127","3,721,525","338,958","199,212","84,057","115,929","261,924","137,630","2,347,932","106,937"
2011,"311,556,874","166,634","703,200","154,545","118,164","872,778","898,016","154,402","824,930","126,970",...,"202,841","3,678,732","335,118","197,379","82,811","113,915","257,719","136,415","2,322,216","104,954"
2012,"313,830,990","167,442","702,109","153,976","118,273","874,698","901,939","154,400","826,144","126,544",...,"202,475","3,634,488","330,752","195,493","82,192","111,845","253,345","135,185","2,295,419","102,970"
2013,"315,993,715","167,473","703,621","152,667","118,405","877,065","904,953","154,623","825,731","125,836",...,"202,420","3,593,077","326,684","194,023","81,430","109,938","249,504","133,955","2,269,908","101,054"
2014,"318,301,008","168,342","704,908","151,949","119,042","878,113","904,538","154,558","828,470","125,274",...,"204,054","3,534,874","321,451","191,124","80,239","107,458","244,760","132,195","2,233,776","98,695"


In [1]:
#use fbprophet to forecast population growth. In our example, y = population, ds = date

import fbprophet

#creat column for dates needed for forecast to work
#census_df.reset_index(level=0,inplace=True)
#census_df

#census_prophet=census_df.rename(columns={})

ModuleNotFoundError: No module named 'fbprophet'

In [6]:
# parallel coordinates plot

file_name='combined_df.csv'
csv_file=Path(f"../Data/Clean/{file_name}")
combined_df= pd.read_csv(csv_file)
combined_df.head()
combined_df.dtypes

MSA                                    object
Violent Crime                         float64
Murder                                float64
Murder Rate                           float64
Population 2019 (DO NOT USE)          float64
Population 2010 (DO NOT USE)          float64
Population 10 year CAGR               float64
Median income (dollars)               float64
Mean income (dollars)                 float64
City                                   object
State                                  object
Population                            float64
Density                               float64
Zip Code                              float64
lat                                   float64
lon                                   float64
% Sale Price Change Since Feb         float64
% Sale Price Change Since Feb 2019    float64
Avg Sale Price Last 12M               float64
Sale Price Last                       float64
Sale Pr CAGR GFC                      float64
Sale Pr Standard Deviation        

In [22]:
# Pulling a subset of data from combined df
# Min Population - leave equal to 0 if no conditions
min_population=0
columns_needed=['MSA',
                'Murder Rate',
                'Median income (dollars)',
                'Capitalization Rate Last',
                'Density',
                '5 Year Avg Return',
                'Sale Price Last',
                'Monthly Rent Last',
                
               ]
data_frame=combined_df[combined_df['Population']>min_population]

# data_frame.set_index('MSA', inplace=True)
paralleldata_frame=data_frame[columns_needed].dropna(subset=columns_needed)

plotparallelcoord=px.parallel_coordinates(
        paralleldata_frame,
        labels={"Median income (dollars)": "Median Income",
                "Capitalization Rate Last": "Cap Rate",
                "sepal_length": "Sepal Length",
                "5 Year Avg Return": "Forecast Return (5yr)",
                "% Sale Price Change Since Feb 2019": "Sale Price Change", 
                "% Rent Change Since Feb 2019": "Rent Price Change"},
        color="Median income (dollars)",
        title="Relationships between various real estate variables")


plotparallelcoord


In [44]:
# scatter matrix for salesprice vs rent 
census_df.head


fig = px.scatter_matrix(combined_df, dimensions=["% Sale Price Change Since Feb", "% Rent Change Since Feb"],)
fig.show()

In [60]:
#TIME SERIES

file_name='house_price_df.csv'
csv_file=Path(f"../Data/Clean/{file_name}")


house_price_df= pd.read_csv(csv_file)
house_price_df=house_price_df[["Date","New York, NY"]]
house_price_df.head()

,Date,"New York, NY"
0,1/31/1996,187842.0
1,2/29/1996,187403.0
2,3/31/1996,187125.0
3,4/30/1996,186592.0
4,5/31/1996,186274.0


In [12]:
fig = px.line(df, x="year", y="lifeExp", color='country')
fig.show()
census_df.head(5)
fig = px.line(census_df,x="Date")
fig.show()

NameError: name 'df' is not defined